In [ ]:
pip install regex

In [ ]:
# 사용된 특수 문자 추출

import os
import re
import pandas as pd

# 특수문자만 추출하는 정규 표현식 패턴 (한글, 영어, 숫자, 한자, 일본어 제외)
special_char_pattern = r'[^가-힣a-zA-Z0-9一-龥ぁ-ゔァ-ヴー々〆〤\s]'

# 폴더 경로와 결과 저장할 엑셀 파일 이름 설정
folder_path = r'경로'  # 폴더 경로로 변경
output_path = '파일명.xlsx'

# 특수문자를 저장할 집합 (중복 제거용)
unique_special_chars = set()

# 폴더 안의 모든 엑셀 파일을 읽어서 특수문자 추출
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(folder_path, filename)
        try:
            # 엑셀 파일 읽기
            xl = pd.ExcelFile(file_path)
            for sheet_name in xl.sheet_names:
                df = xl.parse(sheet_name)
                
                # 모든 셀에 대해 특수문자 추출
                for col in df.columns:
                    df[col] = df[col].astype(str)  # 모든 값을 문자열로 변환
                    for cell in df[col]:
                        special_chars = re.findall(special_char_pattern, cell)
                        unique_special_chars.update(special_chars)  # 집합에 추가하여 중복 제거
        except Exception as e:
            print(f"파일 {filename}에서 오류 발생: {e}")

# 추출한 특수문자를 데이터프레임으로 변환 후 엑셀 파일로 저장
extracted_df = pd.DataFrame({'특수문자': list(unique_special_chars)})
extracted_df.to_excel(output_path, index=False)

print("특수문자 추출 및 중복 제거가 완료되었습니다.")

In [ ]:
# 헷갈리는 특수문자 앞뒤 문맥 추출

import os
import re
import pandas as pd

# 폴더 경로 설정
folder_path = r'경로'  # 폴더 경로로 변경
output_path = '파일명.xlsx'

# 특수문자 목록과 패턴 (정규식 사용을 위해 이스케이프 문자 필요)
special_chars = ['〃']
special_char_pattern = '|'.join([re.escape(char) for char in special_chars])

# 추출 결과 저장을 위한 리스트
extracted_data = []

# 폴더 내 모든 엑셀 파일 처리
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(folder_path, filename)
        try:
            # 엑셀 파일 읽기
            xl = pd.ExcelFile(file_path)
            for sheet_name in xl.sheet_names:
                df = xl.parse(sheet_name)
                
                # 모든 셀에 대해 특수문자 앞뒤 어절 추출
                for col in df.columns:
                    df[col] = df[col].astype(str)  # 모든 값을 문자열로 변환
                    for cell in df[col]:
                        # 특수문자 위치 찾기
                        matches = re.finditer(special_char_pattern, cell)
                        for match in matches:
                            start = max(0, match.start() - 100)  # 특수문자 앞 어절 시작점
                            end = min(len(cell), match.end() + 100)  # 특수문자 뒤 어절 끝점
                            context = cell[start:end]  # 앞뒤 어절 포함한 부분 추출
                            extracted_data.append({
                                '파일명': filename,
                                '시트명': sheet_name,
                                '열': col,
                                '특수문자': match.group(),
                                '문맥': context
                            })
        except Exception as e:
            print(f"파일 {filename}에서 오류 발생: {e}")

# 결과를 데이터프레임으로 변환 후 엑셀 파일로 저장
extracted_df = pd.DataFrame(extracted_data)
extracted_df.to_excel(output_path, index=False)

print("특수문자 앞뒤 문맥 추출이 완료되었습니다.")


In [ ]:
# 특수문자 사이 단어 추출

import pandas as pd
import re
import glob
import os

# 처리할 엑셀 파일들이 있는 폴더 경로 지정
folder_path = r'경로'

# 폴더 내 모든 .xlsx 파일 검색
excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

# 지정된 특수 문자 쌍 정의
special_char_pairs = [
    ("＇", "'"), 
    ('"', '"'), 
    ("(", ")"), 
    ("[", "]"), 
    ("{", "}"), 
    ("<", ">"), 
    ("《", "》"), 
    ("｢", "｣"), 
    ("『", "』"), 
    ("【", "】")
]

# 각 특수 문자 쌍에 대한 정규 표현식 패턴 생성
pattern = re.compile('|'.join([re.escape(o) + r'(.+?)' + re.escape(c) for o, c in special_char_pairs]))

# 추출한 결과를 저장할 세트 (중복 제거)
extracted_data = set()

# 에러 로그 기록용 리스트
error_log = []

for file in excel_files:
    try:
        print(f'{file} 처리 중...')
        
        # 엑셀 파일의 모든 시트 읽기
        xls = pd.ExcelFile(file)
        for sheet_name in xls.sheet_names:
            try:
                df = pd.read_excel(file, sheet_name=sheet_name, dtype=str)
                
                # 모든 셀에서 특수 문자 쌍 포함 텍스트 추출
                for col in df.columns:
                    for cell in df[col].dropna():  # 비어있지 않은 셀에 대해서만 처리
                        # 정규표현식으로 특수문자 쌍 포함 추출
                        matches = pattern.finditer(cell)
                        
                        # 각 매칭 결과를 세트에 추가 (중복 제거)
                        for match in matches:
                            extracted_data.add(match.group(0))  # 특수문자 포함 전체 문자열 추가
            
            except Exception as sheet_error:
                error_log.append(f'파일: {file}, 시트: {sheet_name} 처리 중 에러 발생 - {sheet_error}')
                print(f'파일: {file}, 시트: {sheet_name} 처리 중 에러 발생 - {sheet_error}')

    except Exception as file_error:
        error_log.append(f'파일: {file} 열기 중 에러 발생 - {file_error}')
        print(f'파일: {file} 열기 중 에러 발생 - {file_error}')

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(sorted(extracted_data), columns=["Extracted Text"])

# 시트 최대 행수인 1,048,576 행을 초과하지 않도록 나누어 저장
output_file = '특수문자_사이_단어_포함.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for i in range(0, len(result_df), 1048576):
        result_df.iloc[i:i+1048576].to_excel(writer, sheet_name=f'Sheet_{i // 1048576 + 1}', index=False)

# 에러 로그 출력
if error_log:
    print("\n에러 로그:")
    for log in error_log:
        print(log)
else:
    print("모든 파일이 성공적으로 처리되었습니다.")

print(f'추출 완료! 결과는 {output_file} 파일을 확인하세요.')


In [ ]:
# 작품명 관련 특수 문자 정규화.

import os
import pandas as pd
import re

# 폴더 경로 설정
folder_path = r'경로'

# 대체할 특수문자 매핑 (왼쪽 문자: 오른쪽 문자)
replace_dict = {
    "＇": "'",
    '"': '"',
    "＂": '"',
    "(": "(",
    "（": "(",
    ")": ")",
    "）": ")",
    "[": "[",
    "［": "[",
    "]": "]",
    "］": "]",
    "`": "'", 
    "｀": "'",
    "{": "{",
    "}": "}",
    "´": "'",
    "˝": '"',
    "‘": "'",
    "’": "'",
    "‛": "'",
    "“": '"',
    "”": '"',
    "′": "'",
    "″": '"',
    "〈": "<",
    "〉": ">",
    "《": "《",
    "》": "》",
    "｢": "｢",
    "「": "｢",
    "｣": "｣",
    "」": "｣",
    "󰡔": "『",
    "󰡕": "』",
    "【": "【",
    "】": "】",
    "〔": "[",
    "〕": "]",
    "<": "<",
    "＜": "<",
    ">": ">",
    "＞": ">",
    "≪": "《",
    "≫": "》",
}

# 오류 로그 기록
error_log = []

# 폴더 내 모든 엑셀 파일 처리
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls') or filename.startswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        try:
            # 엑셀 파일 열기
            xl = pd.ExcelFile(file_path, engine='openpyxl')
            
            with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                for sheet_name in xl.sheet_names:
                    try:
                        # 시트 읽기 및 빈 데이터 확인
                        df = xl.parse(sheet_name)
                        if df.empty:
                            print(f"{filename} - 시트 '{sheet_name}'이 비어 있습니다. 건너뜁니다.")
                            continue
                        
                        # 모든 셀에 대해 대체 문자 적용
                        for col in df.columns:
                            try:
                                df[col] = df[col].astype(str).replace("nan", "")
                                for old_char, new_char in replace_dict.items():
                                    try:
                                        # 대체 적용 및 오류 처리
                                        df[col] = df[col].str.replace(re.escape(old_char), new_char, regex=True)
                                    except Exception as cell_error:
                                        error_log.append(f"'{filename}', 시트 '{sheet_name}', 열 '{col}'에서 '{old_char}' 대체 중 오류: {cell_error}")
                                        continue
                            except Exception as col_error:
                                error_log.append(f"'{filename}', 시트 '{sheet_name}', 열 '{col}' 처리 중 오류: {col_error}")
                                continue
                        
                        # 변경 내용을 파일에 저장
                        df.to_excel(writer, sheet_name=sheet_name, index=False)
                        print(f"{filename} - 시트 '{sheet_name}' 처리 완료.")

                    except Exception as sheet_error:
                        error_log.append(f"'{filename}' 파일의 시트 '{sheet_name}' 처리 중 오류: {sheet_error}")
                        continue

        except PermissionError:
            error_log.append(f"'{filename}' 파일에서 권한 오류 발생: 다른 프로그램에서 파일이 열려 있을 수 있습니다.")
        except Exception as e:
            error_log.append(f"'{filename}' 파일 열기 중 오류 발생: {e}")

print("처리가 완료되었습니다. 다음은 발생한 오류 목록입니다(있을 경우):")
for error in error_log:
    print(error)

In [ ]:
# 특수문자 정규화 200행씩 분할. 동아일보

import os
import pandas as pd
import re

# 폴더 경로 설정 (예시 경로)
folder_path = r'경로'

replace_dict = {
    "＇": "'",
    '"': '"',
    "＂": '"',
    "(": "(",
    "（": "(",
    ")": ")",
    "）": ")",
    "[": "[",
    "［": "[",
    "]": "]",
    "］": "]",
    "`": "'", 
    "｀": "'",
    "{": "{",
    "}": "}",
    "´": "'",
    "˝": '"',
    "‘": "'",
    "’": "'",
    "‛": "'",
    "“": '"',
    "”": '"',
    "′": "'",
    "″": '"',
    "〈": "<",
    "〉": ">",
    "《": "《",
    "》": "》",
    "｢": "｢",
    "「": "｢",
    "｣": "｣",
    "」": "｣",
    "󰡔": "『",
    "󰡕": "』",
    "【": "【",
    "】": "】",
    "〔": "[",
    "〕": "]",
    "<": "<",
    "＜": "<",
    ">": ">",
    "＞": ">",
    "≪": "《",
    "≫": "》",
}

# 병합할 데이터프레임 초기화
final_merged_df = pd.DataFrame()

# 오류 로그
error_log = []

# 각 파일에 대해 처리
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls') or filename.startswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        try:
            xl = pd.ExcelFile(file_path, engine='openpyxl')
            
            for sheet_name in xl.sheet_names:
                try:
                    df = xl.parse(sheet_name)
                    if df.empty:
                        print(f"{filename} - 시트 '{sheet_name}'이 비어 있습니다. 건너뜁니다.")
                        continue
                    
                    # 200행씩 분할하여 처리
                    num_splits = (len(df) // 200) + 1
                    for i in range(num_splits):
                        split_df = df.iloc[i * 200 : (i + 1) * 200].copy()

                        # 각 셀에 대체 문자 적용
                        for col in split_df.columns:
                            try:
                                split_df.loc[:, col] = split_df[col].astype(str).replace("nan", "")
                                for old_char, new_char in replace_dict.items():
                                    try:
                                        split_df.loc[:, col] = split_df[col].str.replace(re.escape(old_char), new_char, regex=True)
                                    except Exception as cell_error:
                                        error_log.append(f"'{filename}', 시트 '{sheet_name}', 열 '{col}'에서 '{old_char}' 대체 중 오류: {cell_error}")
                                        continue
                            except Exception as col_error:
                                error_log.append(f"'{filename}', 시트 '{sheet_name}', 열 '{col}' 처리 중 오류: {col_error}")
                                continue

                        # 처리된 200행을 최종 병합 데이터프레임에 추가
                        final_merged_df = pd.concat([final_merged_df, split_df], ignore_index=True)

                except Exception as sheet_error:
                    error_log.append(f"'{filename}' 파일의 시트 '{sheet_name}' 처리 중 오류: {sheet_error}")
                    continue

        except PermissionError:
            error_log.append(f"'{filename}' 파일에서 권한 오류 발생: 다른 프로그램에서 파일이 열려 있을 수 있습니다.")
        except Exception as e:
            error_log.append(f"'{filename}' 파일 열기 중 오류 발생: {e}")

# 모든 처리가 끝난 후 최종 병합된 파일 저장
merged_file_path = os.path.join(folder_path, "final_merged_output.xlsx")
final_merged_df.to_excel(merged_file_path, index=False)
print(f"최종 병합 파일이 '{merged_file_path}'에 저장되었습니다.")

# 오류 로그 출력
print("처리가 완료되었습니다. 다음은 발생한 오류 목록입니다(있을 경우):")
for error in error_log:
    print(error) 

